[![Open In Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/tushar-mahalya/Forecasting-using-Bidirectional-LSTM/blob/master/model_selector.ipynb)

In [1]:
# Hardware configuration 
import tensorflow as tf
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 13205704365003700418
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14444920832
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3632835233995326412
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:1e.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
# Importing important pakages
import os
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import OrderedDict
from termcolor import colored as color

# Importing TF.keras packages
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.layers import Dense, LSTM, Dropout, Bidirectional

# To avoid irrelevent warning messages
import logging
logging.getLogger('tensorflow').disabled = True
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
# Yahoo Finance API
import yfinance

# Function to download stock data (univariate or multivariate)
from Data_Downloader import data_accumilator

In [4]:
# Importing custom defined models into local runtime
from Models import models
my_models = models()
my_models

{'Fixed': <function Models.simple_model(X, y, learn_rate, model_name=None)>,
 'Dynamic': <function Models.dynamic_model(X, y, learn_rate, model_name=None)>,
 'Bidirectional': <function Models.bidirectional_model(X, y, learn_rate, model_name=None)>,
 'Stacked': <function Models.stacked_model(X, y, learn_rate, model_name=None)>}

In [5]:
# Nifty50 stocks indormation
nifty50 = pd.read_csv('nifty50.csv')
nifty50.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Enterprises Ltd.,Metals & Mining,ADANIENT,EQ,INE423A01024
1,Adani Ports and Special Economic Zone Ltd.,Services,ADANIPORTS,EQ,INE742F01042
2,Apollo Hospitals Enterprise Ltd.,Healthcare,APOLLOHOSP,EQ,INE437A01024
3,Asian Paints Ltd.,Consumer Durables,ASIANPAINT,EQ,INE021A01026
4,Axis Bank Ltd.,Financial Services,AXISBANK,EQ,INE238A01034


In [6]:
# Get the current working directory (cwd)
cwd = os.getcwd()

# Downloading Nifty50 stocks data in Current working directory
preprocessed_data = data_accumilator(cwd, nifty50['Symbol'])

Directory named "Stocks Data" with required data is downloaded successfully at "/home/studio-lab-user/Forecasting-using-Bidirectional-LSTM"


In [7]:
# Dictionary with stock ticker(key) and location of 
# normalized data of that stock (value)
dict(list(preprocessed_data.items())[:5])  # First 5 elements

{'ADANIENT': '/home/studio-lab-user/Forecasting-using-Bidirectional-LSTM/Stocks Data/ADANIENT/Normalized_ADANIENT.csv',
 'ADANIPORTS': '/home/studio-lab-user/Forecasting-using-Bidirectional-LSTM/Stocks Data/ADANIPORTS/Normalized_ADANIPORTS.csv',
 'APOLLOHOSP': '/home/studio-lab-user/Forecasting-using-Bidirectional-LSTM/Stocks Data/APOLLOHOSP/Normalized_APOLLOHOSP.csv',
 'ASIANPAINT': '/home/studio-lab-user/Forecasting-using-Bidirectional-LSTM/Stocks Data/ASIANPAINT/Normalized_ASIANPAINT.csv',
 'AXISBANK': '/home/studio-lab-user/Forecasting-using-Bidirectional-LSTM/Stocks Data/AXISBANK/Normalized_AXISBANK.csv'}

In [8]:
# Importing data initializing function
from Data_Initializer import train_test_data

# Importing best model selector function
from Model_Selector import test_models, save_models

## Selecting Best Model

In [9]:
# Selecting hyperparameters to test on data
window_size_list = [5,7,10,14]
epochs = 100

In [10]:
params_dict, best_params = test_models(data_dict = preprocessed_data,
                                       epochs = epochs,
                                       models = my_models,
                                       window_sizes = window_size_list)


================================ NSE : ADANIENT ================================

--------------------------------------------------------------- Simple Sequence

Window size : 5
----------------
  > Model: Fixed           Param count: 146 	Training error: 0.0013	Testing error: 0.1441
  > Model: Dynamic         Param count: 146 	Training error: 0.0015	Testing error: 0.1629
  > Model: Bidirectional   Param count: 341 	Training error: 0.0013	Testing error: 0.1047
  > Model: Stacked         Param count: 806 	Training error: 0.0016	Testing error: 0.1063

Window size : 7
----------------
  > Model: Fixed           Param count: 146 	Training error: 0.0011	Testing error: 0.0642
  > Model: Dynamic         Param count: 260 	Training error: 0.0015	Testing error: 0.1250
  > Model: Bidirectional   Param count: 617 	Training error: 0.0013	Testing error: 0.0758
  > Model: Stacked         Param count: 806 	Training error: 0.0019	Testing error: 0.1158

Window size : 10
----------------
  > Model: Fix

## Saving Best Performing Models

In [11]:
save_models(data_dict = preprocessed_data,
            epochs = 500,
            models = my_models,
            best_params = best_params)


(1) NSE:ADANIENT
-----------------------
 >> Model : Bidirectional	 >> Window Size : 14
 >> Seq. Type : Simple Sequence	 >> Epochs : 500
 >> Training Error : 0.0006	 >> Testing Error : 0.0092

Succesfully saved best performing models in 'Models' directory.

(2) NSE:ADANIPORTS
-----------------------
 >> Model : Dynamic	 >> Window Size : 14
 >> Seq. Type : Simple Sequence	 >> Epochs : 500
 >> Training Error : 0.0048	 >> Testing Error : 0.0117

Succesfully saved best performing models in 'Models' directory.

(3) NSE:APOLLOHOSP
-----------------------
 >> Model : Dynamic	 >> Window Size : 7
 >> Seq. Type : Multi Sequence	 >> Epochs : 500
 >> Training Error : 0.0033	 >> Testing Error : 0.0022

Succesfully saved best performing models in 'Models' directory.

(4) NSE:ASIANPAINT
-----------------------
 >> Model : Bidirectional	 >> Window Size : 10
 >> Seq. Type : Simple Sequence	 >> Epochs : 500
 >> Training Error : 0.0058	 >> Testing Error : 0.0039

Succesfully saved best performing models